# 1



```python
import pandas as pd
import numpy as np
```
- **Importazione delle librerie**: Qui importiamo `pandas` per gestire i dati in formato tabellare (DataFrame) e `numpy` per operazioni numeriche come il calcolo delle medie o dei valori di spostamento.

```python
# DataFrame iniziale
data = {
    "date": [
        "2023-05-01", "2023-05-02",
        "2023-05-05", "2023-05-07",
        "2023-05-10", "2023-05-15",
        "2023-05-20", "2023-05-25",
        "2023-05-30", "2023-05-31"
    ],
    "sales_amount": [100, 150, 200, 130, 180, 250, 300, 220, 270, 310]
}
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])  # Convertiamo la colonna 'date' in formato datetime
```
- **Creazione del DataFrame**: Creiamo un dizionario `data` con le colonne `date` e `sales_amount` che contiene rispettivamente le date delle transazioni e l'importo delle vendite.
- **Creazione del DataFrame**: Usiamo `pd.DataFrame(data)` per trasformare il dizionario in un DataFrame.
- **Conversione in formato datetime**: Convertiamo la colonna `date` in formato `datetime` con `pd.to_datetime()`, per permettere operazioni di data come il calcolo di differenze temporali.

```python
# --- 1. Aggiungere una colonna con il numero di giorni trascorsi dalle ultime vendite
df['days_since_last_sale'] = (df['date'].max() - df['date']).dt.days
```
- **Calcolo dei giorni trascorsi dalle ultime vendite**: 
  - `df['date'].max()` restituisce la data più recente nel DataFrame.
  - `(df['date'].max() - df['date'])` calcola la differenza tra la data più recente e tutte le date nel DataFrame.
  - `.dt.days` converte la differenza in giorni.
  - Creiamo una nuova colonna `days_since_last_sale` che contiene il numero di giorni trascorsi dalla vendita più recente per ogni riga.

```python
# --- 2. Aggiungere le date mancanti e riempire con 0 i valori di sales_amount per queste date
# Creiamo una serie di tutte le date tra la prima e l'ultima data
all_dates = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
```
- **Creazione delle date mancanti**: 
  - `pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')` crea un intervallo di tutte le date (con frequenza giornaliera) tra la data minima e massima nel DataFrame.
  
```python
# Creiamo un nuovo DataFrame con tutte le date e uniamo con il DataFrame originale
df_full = pd.DataFrame({'date': all_dates})
df_full = df_full.merge(df, on='date', how='left').fillna({'sales_amount': 0})  # Riempie i valori mancanti con 0
```
- **Creazione del DataFrame con tutte le date**: Creiamo un nuovo DataFrame `df_full` che contiene tutte le date.
- **Unione dei DataFrame**: Usiamo `.merge()` per unire `df_full` con il DataFrame originale `df` sulla colonna `date`, mantenendo tutte le date (anche quelle mancanti nel DataFrame originale) con `how='left'`.
- **Riempimento dei valori mancanti**: Con `.fillna({'sales_amount': 0})` sostituiamo i valori mancanti di `sales_amount` con `0` per le date che non erano nel DataFrame originale.

```python
# --- 3. Creare una colonna con l'importo delle vendite di 7 giorni prima
df_full['sales_7_days_ago'] = df_full['sales_amount'].shift(7, fill_value=0)
```
- **Vendite di 7 giorni fa**: 
  - `.shift(7)` sposta i valori di `sales_amount` di 7 posizioni verso il basso, ottenendo così il valore delle vendite di 7 giorni prima.
  - `fill_value=0` assicura che le prime 7 righe, che non hanno dati di vendita precedenti, vengano riempite con `0`.

```python
# --- 4. Creare una colonna con l'importo medio delle vendite dei 7 giorni precedenti
df_full['avg_sales_7_days'] = df_full['sales_amount'].rolling(window=7, min_periods=1).mean()
```
- **Media mobile delle vendite negli ultimi 7 giorni**:
  - `.rolling(window=7)` crea una finestra mobile di 7 giorni.
  - `.mean()` calcola la media dei valori in questa finestra.
  - `min_periods=1` assicura che la media venga calcolata anche se ci sono meno di 7 giorni (ad esempio, per i primi giorni del DataFrame).

```python
# --- 5. Convertire la frequenza in settimanale, sommando le vendite per ogni settimana
df_full.set_index('date', inplace=True)
df_weekly = df_full.resample('W').agg({'sales_amount': 'sum'})  # Somma delle vendite per ogni settimana
```
- **Impostare la colonna 'date' come indice**: Con `df_full.set_index('date', inplace=True)` rendiamo la colonna `date` l'indice del DataFrame, che ci permette di raggruppare i dati per data.
- **Raggruppamento settimanale**: Con `df_full.resample('W')` raggruppiamo i dati per settimana (`'W'` sta per settimanale).
  - `.agg({'sales_amount': 'sum'})` somma i valori di `sales_amount` per ogni settimana.

```python
# Visualizzare i risultati
print(df_full.head(15))  # Mostra i primi 15 record del DataFrame con tutte le operazioni
print(df_weekly)  # Mostra la somma settimanale delle vendite
```
- **Visualizzazione dei risultati**: 
  - `df_full.head(15)` mostra i primi 15 record del DataFrame con tutte le colonne aggiuntive.
  - `df_weekly` mostra la somma settimanale delle vendite.

### Riassunto:
1. Aggiungiamo una colonna che calcola i giorni trascorsi dalle ultime vendite.
2. Creiamo un DataFrame con tutte le date mancanti, riempiamo con 0 i valori di `sales_amount` per queste date.
3. Creiamo una colonna con le vendite di 7 giorni prima.
4. Calcoliamo la media delle vendite degli ultimi 7 giorni.
5. Raggruppiamo le vendite per settimana e sommiamo i valori.

Se c'è qualcosa che non è chiaro o se vuoi un approfondimento su un passaggio specifico, fammi sapere!

# 2

# OPPURE 



```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing
```
1. **Importazione delle librerie**:
   - `pandas` (pd): per la gestione dei dati in formato DataFrame.
   - `numpy` (np): per operazioni matematiche avanzate, in questo caso per generare i dati delle vendite.
   - `matplotlib.pyplot` (plt): per la creazione di grafici.
   - `seaborn` (sns): per una visualizzazione più avanzata dei grafici.
   - `ExponentialSmoothing` da `statsmodels.tsa.holtwinters`: per applicare il modello Holt-Winters per il smoothing esponenziale.

```python
date_range = pd.date_range(start="2023-01-01", periods=180, freq="D")
```
2. **Creazione della sequenza di date**:
   - `pd.date_range` genera una sequenza di date che parte dal 1° gennaio 2023 (`start="2023-01-01"`) e dura 180 giorni (`periods=180`), con frequenza giornaliera (`freq="D"`). Il risultato è una serie di date giornaliere.

```python
values = np.array([200, 220, 250, 230, 240, 260, 280], dtype=float)
```
3. **Creazione dei valori delle vendite**:
   - `np.array(...)` crea un array di numeri che rappresentano i valori delle vendite per 7 giorni. Questi valori sono ipotetici e si ripetono ciclicamente ogni settimana.

```python
sales = np.tile(values, len(date_range) // 7 + 1)[:len(date_range)]
```
4. **Ripetizione dei valori delle vendite per 180 giorni**:
   - `np.tile(values, len(date_range) // 7 + 1)` ripete l'array `values` per adattarlo alla lunghezza della sequenza di date. La divisione per 7 è necessaria per determinare quante settimane ci sono nei 180 giorni.
   - `[:len(date_range)]` fa sì che l'array risultante abbia la stessa lunghezza della sequenza di date (180 giorni).

```python
sales += np.random.normal(0, 10, len(sales))
```
5. **Aggiunta di rumore casuale ai dati delle vendite**:
   - `np.random.normal(0, 10, len(sales))` genera una distribuzione normale con media 0 e deviazione standard 10, per simulare la variabilità casuale nelle vendite.
   - Il rumore viene aggiunto ai dati delle vendite esistenti.

```python
df = pd.DataFrame({"date": date_range, "sales": sales})
df.set_index("date", inplace=True)
```
6. **Creazione del DataFrame**:
   - `pd.DataFrame(...)` crea un DataFrame con due colonne: "date" (le date generate precedentemente) e "sales" (le vendite con il rumore aggiunto).
   - `df.set_index("date", inplace=True)` imposta la colonna "date" come indice del DataFrame, il che rende il lavoro con i dati temporali più facile per analisi e modelli.

```python
model = ExponentialSmoothing(df['sales'], trend='add', seasonal='add', seasonal_periods=7)
```
7. **Creazione del modello Holt-Winters**:
   - `ExponentialSmoothing(df['sales'], ...)` crea un modello di smoothing esponenziale basato sui dati storici delle vendite (`df['sales']`).
   - `trend='add'`: indica che si prevede un trend additivo (ovvero un incremento o decremento costante delle vendite nel tempo).
   - `seasonal='add'`: significa che c'è una componente stagionale additiva (ad esempio, vendite più alte ogni settimana).
   - `seasonal_periods=7`: indica che c'è una stagionalità settimanale (ogni 7 giorni).

```python
hw_fit = model.fit()
```
8. **Adattamento del modello ai dati**:
   - `model.fit()` esegue il fitting del modello sui dati storici. Il modello cerca di trovare i parametri ottimali per il trend e la stagionalità per meglio adattarsi ai dati passati.

```python
forecast = hw_fit.forecast(steps=7)
```
9. **Previsioni per i prossimi 7 giorni**:
   - `hw_fit.forecast(steps=7)` utilizza il modello addestrato per fare previsioni per i prossimi 7 giorni. Restituisce un array con i valori previsti delle vendite.

```python
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x=df.index, y="sales", label='Vendite Storiche')
```
10. **Visualizzazione delle vendite storiche**:
    - `plt.figure(figsize=(12, 6))` imposta la dimensione della figura del grafico.
    - `sns.lineplot(...)` crea un grafico a linea delle vendite storiche. `df.index` rappresenta le date e `"sales"` i valori delle vendite.

```python
plt.plot(pd.date_range(df.index[-1], periods=8, freq='D')[1:], forecast, label='Previsioni (7 giorni)', linestyle='--', color='red')
```
11. **Visualizzazione delle previsioni**:
    - `pd.date_range(df.index[-1], periods=8, freq='D')` crea una nuova sequenza di date a partire dall'ultimo giorno delle vendite storiche (`df.index[-1]`), per i successivi 8 giorni (compreso il giorno corrente).
    - `[1:]` rimuove il primo giorno (che è già incluso tra le vendite storiche).
    - `plt.plot(...)` traccia una linea rossa tratteggiata (`linestyle='--'`) con le previsioni per i prossimi 7 giorni.

```python
plt.title("Vendite Giornaliere con Previsioni a 7 Giorni")
plt.xlabel("Data")
plt.ylabel("Vendite")
plt.legend()
plt.show()
```
12. **Impostazioni del grafico**:
    - `plt.title(...)` imposta il titolo del grafico.
    - `plt.xlabel(...)` e `plt.ylabel(...)` impostano le etichette degli assi.
    - `plt.legend()` mostra la legenda del grafico.
    - `plt.show()` visualizza il grafico.

```python
print(f"Previsioni per i prossimi 7 giorni:\n{forecast}")
```
13. **Stampa delle previsioni**:
    - Stampa i valori previsti per i prossimi 7 giorni.

### In sintesi:
Il codice crea un modello Holt-Winters per prevedere le vendite future basate su dati storici, aggiungendo rumore per simulare fluttuazioni reali. Poi, utilizza il modello per fare previsioni per i prossimi 7 giorni e visualizza il risultato con un grafico.